In [1]:
import time
import torch
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [3]:
# Path to your trained model checkpoint
model_path = "query_classification/checkpoint-420"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

model.to(device)
model.eval()


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [4]:
test_csv = "val_data_all.csv"

df = pd.read_csv(test_csv)
df.head()


,id,annotator,gpt测试结果,答案,Unnamed: 4,position,question,table_file,answer_coordinates,answer_text,aggregation_label,float_answer,Unnamed: 9,澶囨敞aggregation_label涓細\n0锛歂ONE\n1锛歋UM\n2锛欰VERAGE\n3: COUNT,label
0,NaN,NaN,1.0,NaN,NaN,NaN,What is the elevation of Ground in meters abov...,52Theparade(rvt2017)_floor.csv,"['(1, 2)']",['-46.5370539649164'],0,-46.537054,NaN,NaN,floor
1,NaN,NaN,0.0,167.000,产生幻觉,NaN,"How many windows are in the building, do you k...",161210Med_Dent_Clinic_Combined_floor.csv,"['(8, 7)', '(9, 7)', '(13, 7)']",['94.0'],1,94.000000,NaN,NaN,floor
2,NaN,NaN,1.0,NaN,NaN,NaN,What is the number of stairs on EG?,20170601_Mauer_BmB_floor.csv,"['(2, 4)']",['1.0'],0,1.000000,NaN,NaN,floor
3,NaN,NaN,1.0,NaN,NaN,NaN,How many stairs are there in 4th Floor?,OfficeBuilding_floor.csv,"['(2, 5)']",['5.0'],0,5.000000,NaN,NaN,floor
4,NaN,NaN,0.0,2.965,NaN,NaN,How high above the ground is Level 7 in relati...,Learningzonecorentin_floor.csv,"['(8, 2)']",['6.823'],0,6.823000,NaN,NaN,floor


In [5]:
texts = df["question"].tolist()
labels = df["label"].tolist()

len(texts), len(labels)


(348, 348)

In [10]:
correct = 0
total = len(texts)

for text, true_label in zip(texts, labels):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)

    pred_id = torch.argmax(outputs.logits, dim=1).item()
    pred_label = model.config.id2label[pred_id]

    if pred_label == true_label:
        correct += 1

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.9914
